In [ ]:
import os
import torch
import detectron2

from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data import DatasetMapper

# ----------------------------------------------------------
# 1. Register Dataset
# ----------------------------------------------------------
VAL_DATASET_NAME = "neu_valid"
VAL_JSON = "<Domain_A_evaluation_data_annotations_json_file>"
VAL_IMG_DIR = "<Domain_A_evaluation_data_images>"

# Register once
try:
    register_coco_instances(VAL_DATASET_NAME, {}, VAL_JSON, VAL_IMG_DIR)
except:
    pass

# ----------------------------------------------------------
# 2. Configuration
# ----------------------------------------------------------
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(
    "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
))

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4
cfg.DATALOADER.NUM_WORKERS = 2

cfg.OUTPUT_DIR = "/home/aamena/Assessment_tcs/output/domain_a_baseline"
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

# ----------------------------------------------------------
# 3. Evaluator + Loader
# ----------------------------------------------------------
predictor = DefaultPredictor(cfg)
test_mapper = DatasetMapper(cfg, is_train=False)

# FIXED: correct arguments
eval_dataloader = build_detection_test_loader(
    cfg,
    VAL_DATASET_NAME,
    mapper=test_mapper
)

evaluator = COCOEvaluator(
    VAL_DATASET_NAME,
    cfg,
    False,
    output_dir=os.path.join(cfg.OUTPUT_DIR, "validation_results")
)

# ----------------------------------------------------------
# 4. Run Evaluation
# ----------------------------------------------------------
print("🚀 Starting evaluation...")

results = inference_on_dataset(
    predictor.model,
    eval_dataloader,
    evaluator
)

print("\n================================")
print(" FINAL EVALUATION RESULTS")
print("================================")
print(results)

/home/aamena/Assessment_tcs/.venv/lib/python3.10/site-packages/detectron2/model_zoo/model_zoo.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


[11/19 17:54:15 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /home/aamena/Assessment_tcs/output/domain_a_baseline/model_final.pth ...
[11/19 17:54:15 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
WARNING [11/19 17:54:15 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[11/19 17:54:15 d2.data.datasets.coco]: Loaded 1728 images in COCO format from datasets/NEU-seg-DOMAIN_A2/annotations.coco.json
[11/19 17:54:15 d2.data.build]: Distribution of instances among all 4 categories:
|   category    | #instances   |  category  | #instances   |  category  | #instances   |
|:-------------:|:-------------|:----------:|:-------------|:----------:|:-------------|
| In-"Pa",-"Sc" | 0            |     In     | 1705         |     Pa     | 1240         |
|      Sc       | 1162         |           

/home/aamena/Assessment_tcs/.venv/lib/python3.10/site-packages/torch/functional.py:505: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4317.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
W1119 17:54:15.744000 75665 torch/fx/_symbolic_trace.py:52] is_fx_tracing will return true for both fx.symbolic_trace and torch.export. Please use is_fx_tracing_symbolic_tracing() for specifically fx.symbolic_trace or torch.compiler.is_compiling() for specifically torch.export/compile.


[11/19 17:54:16 d2.evaluation.evaluator]: Inference done 11/1728. Dataloading: 0.0006 s/iter. Inference: 0.0592 s/iter. Eval: 0.0014 s/iter. Total: 0.0612 s/iter. ETA=0:01:45
[11/19 17:54:21 d2.evaluation.evaluator]: Inference done 92/1728. Dataloading: 0.0008 s/iter. Inference: 0.0597 s/iter. Eval: 0.0016 s/iter. Total: 0.0621 s/iter. ETA=0:01:41
[11/19 17:54:26 d2.evaluation.evaluator]: Inference done 173/1728. Dataloading: 0.0009 s/iter. Inference: 0.0598 s/iter. Eval: 0.0015 s/iter. Total: 0.0621 s/iter. ETA=0:01:36
[11/19 17:54:31 d2.evaluation.evaluator]: Inference done 254/1728. Dataloading: 0.0009 s/iter. Inference: 0.0598 s/iter. Eval: 0.0015 s/iter. Total: 0.0622 s/iter. ETA=0:01:31
[11/19 17:54:36 d2.evaluation.evaluator]: Inference done 335/1728. Dataloading: 0.0009 s/iter. Inference: 0.0598 s/iter. Eval: 0.0015 s/iter. Total: 0.0622 s/iter. ETA=0:01:26
[11/19 17:54:41 d2.evaluation.evaluator]: Inference done 416/1728. Dataloading: 0.0009 s/iter. Inference: 0.0599 s/iter. E

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# ------------------------------------------
# Convert Detectron2 COCO results → DataFrames
# ------------------------------------------
bbox_results = results["bbox"]
segm_results = results["segm"]

bbox_df = pd.DataFrame(bbox_results, index=[0]).T.rename(columns={0: "Score"})
segm_df = pd.DataFrame(segm_results, index=[0]).T.rename(columns={0: "Score"})

print("📌 Bounding Box Metrics")
display(bbox_df)

print("\n📌 Segmentation Metrics")
display(segm_df)

📌 Bounding Box Metrics


,Score
AP,60.292030
AP50,91.753071
AP75,66.565830
APs,18.229975
APm,46.454343
APl,68.496594
"AP-In-""Pa"",-""Sc""",NaN
AP-In,53.122025
AP-Pa,74.302181
AP-Sc,53.451885



📌 Segmentation Metrics


,Score
AP,48.786723
AP50,86.491578
AP75,50.122322
APs,7.518243
APm,34.628621
APl,55.978872
"AP-In-""Pa"",-""Sc""",NaN
AP-In,43.656772
AP-Pa,67.339760
AP-Sc,35.363637


In [ ]:
import os
import torch
import detectron2

from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data import DatasetMapper

# ----------------------------------------------------------
# 1. Register Dataset
# ----------------------------------------------------------
VAL_DATASET_NAME = "neu_valid"
VAL_JSON = "datasets/NEU-seg-DOMAIN_B2OR/annotations.coco.json"
VAL_IMG_DIR = "<Domain_B_Original_evaluation_data_images>"

# Register once
try:
    register_coco_instances(VAL_DATASET_NAME, {}, VAL_JSON, VAL_IMG_DIR)
except:
    pass

# ----------------------------------------------------------
# 2. Configuration
# ----------------------------------------------------------
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(
    "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
))

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4
cfg.DATALOADER.NUM_WORKERS = 2

cfg.OUTPUT_DIR = "/home/aamena/Assessment_tcs/output/domain_a_baseline"
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

# ----------------------------------------------------------
# 3. Evaluator + Loader
# ----------------------------------------------------------
predictor = DefaultPredictor(cfg)
test_mapper = DatasetMapper(cfg, is_train=False)

# FIXED: correct arguments
eval_dataloader = build_detection_test_loader(
    cfg,
    VAL_DATASET_NAME,
    mapper=test_mapper
)

evaluator = COCOEvaluator(
    VAL_DATASET_NAME,
    cfg,
    False,
    output_dir=os.path.join(cfg.OUTPUT_DIR, "validation_results")
)

# ----------------------------------------------------------
# 4. Run Evaluation
# ----------------------------------------------------------
print("🚀 Starting evaluation...")

results = inference_on_dataset(
    predictor.model,
    eval_dataloader,
    evaluator
)

print("\n================================")
print(" FINAL EVALUATION RESULTS")
print("================================")
print(results)


/home/aamena/Assessment_tcs/.venv/lib/python3.10/site-packages/detectron2/model_zoo/model_zoo.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


[11/19 16:32:07 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /home/aamena/Assessment_tcs/output/domain_a_baseline/model_final.pth ...
[11/19 16:32:07 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
WARNING [11/19 16:32:07 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[11/19 16:32:07 d2.data.datasets.coco]: Loaded 1728 images in COCO format from datasets/NEU-seg-DOMAIN_B2OR/annotations.coco.json
[11/19 16:32:07 d2.data.build]: Distribution of instances among all 4 categories:
|   category    | #instances   |  category  | #instances   |  category  | #instances   |
|:-------------:|:-------------|:----------:|:-------------|:----------:|:-------------|
| In-"Pa",-"Sc" | 0            |     In     | 1705         |     Pa     | 1240         |
|      Sc       | 1162         |         

/home/aamena/Assessment_tcs/.venv/lib/python3.10/site-packages/torch/functional.py:505: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4317.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
W1119 16:32:08.242000 52009 torch/fx/_symbolic_trace.py:52] is_fx_tracing will return true for both fx.symbolic_trace and torch.export. Please use is_fx_tracing_symbolic_tracing() for specifically fx.symbolic_trace or torch.compiler.is_compiling() for specifically torch.export/compile.


[11/19 16:32:09 d2.evaluation.evaluator]: Inference done 11/1728. Dataloading: 0.0006 s/iter. Inference: 0.0605 s/iter. Eval: 0.0010 s/iter. Total: 0.0621 s/iter. ETA=0:01:46
[11/19 16:32:14 d2.evaluation.evaluator]: Inference done 92/1728. Dataloading: 0.0008 s/iter. Inference: 0.0605 s/iter. Eval: 0.0011 s/iter. Total: 0.0624 s/iter. ETA=0:01:42
[11/19 16:32:19 d2.evaluation.evaluator]: Inference done 172/1728. Dataloading: 0.0008 s/iter. Inference: 0.0605 s/iter. Eval: 0.0011 s/iter. Total: 0.0625 s/iter. ETA=0:01:37
[11/19 16:32:24 d2.evaluation.evaluator]: Inference done 252/1728. Dataloading: 0.0009 s/iter. Inference: 0.0606 s/iter. Eval: 0.0011 s/iter. Total: 0.0626 s/iter. ETA=0:01:32
[11/19 16:32:29 d2.evaluation.evaluator]: Inference done 332/1728. Dataloading: 0.0009 s/iter. Inference: 0.0606 s/iter. Eval: 0.0011 s/iter. Total: 0.0626 s/iter. ETA=0:01:27
[11/19 16:32:34 d2.evaluation.evaluator]: Inference done 412/1728. Dataloading: 0.0009 s/iter. Inference: 0.0607 s/iter. E

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# ------------------------------------------
# Convert Detectron2 COCO results → DataFrames
# ------------------------------------------
bbox_results = results["bbox"]
segm_results = results["segm"]

bbox_df = pd.DataFrame(bbox_results, index=[0]).T.rename(columns={0: "Score"})
segm_df = pd.DataFrame(segm_results, index=[0]).T.rename(columns={0: "Score"})

print("📌 Bounding Box Metrics")
display(bbox_df)

print("\n📌 Segmentation Metrics")
display(segm_df)

📌 Bounding Box Metrics


,Score
AP,42.169340
AP50,70.484485
AP75,43.284197
APs,3.212353
APm,29.101541
APl,49.062672
"AP-In-""Pa"",-""Sc""",NaN
AP-In,25.067114
AP-Pa,61.327617
AP-Sc,40.113289



📌 Segmentation Metrics


,Score
AP,34.469904
AP50,65.993048
AP75,32.467856
APs,1.976825
APm,21.267555
APl,40.740903
"AP-In-""Pa"",-""Sc""",NaN
AP-In,20.807488
AP-Pa,57.284944
AP-Sc,25.317280


In [ ]:
import json
import os
from PIL import Image

images_dir = "/home/aamena/Assessment_tcs/datasets/DomainB_Enhanced/images"
ann_path = "/home/aamena/Assessment_tcs/datasets/DomainB_Enhanced/annotations.coco.json"
output_ann = "/home/aamena/Assessment_tcs/datasets/DomainB_Enhanced/annotations_fixed.coco.json"

# Load annotation
with open(ann_path, "r") as f:
    coco = json.load(f)

# Build map of filename → size
size_map = {}
for fname in os.listdir(images_dir):
    if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
        continue
    path = os.path.join(images_dir, fname)
    w, h = Image.open(path).size
    size_map[fname] = (w, h)

updated = 0

# Fix annotation sizes
for img in coco["images"]:
    fname = img["file_name"]
    if fname in size_map:
        w, h = size_map[fname]
        img["width"] = w
        img["height"] = h
        updated += 1

print(f"Updated {updated} image entries.")

# Save new JSON
with open(output_ann, "w") as f:
    json.dump(coco, f, indent=2)

print("Saved:", output_ann)


Updated 1728 image entries.
Saved: /home/aamena/Assessment_tcs/datasets/DomainB_Enhanced/annotations_fixed.coco.json


In [2]:
import os
import torch
import detectron2

from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data import DatasetMapper

# ----------------------------------------------------------
# 1. Register Dataset
# ----------------------------------------------------------
VAL_DATASET_NAME = "neu_valid"
VAL_JSON = "datasets/DomainB_Enhanced/annotations.coco.json"
VAL_IMG_DIR = "datasets/DomainB_Enhanced/images"

# Register once
try:
    register_coco_instances(VAL_DATASET_NAME, {}, VAL_JSON, VAL_IMG_DIR)
except:
    pass

# ----------------------------------------------------------
# 2. Configuration
# ----------------------------------------------------------
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(
    "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
))

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4
cfg.DATALOADER.NUM_WORKERS = 2

cfg.OUTPUT_DIR = "/home/aamena/Assessment_tcs/output/domain_a_baseline"
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

# ----------------------------------------------------------
# 3. Evaluator + Loader
# ----------------------------------------------------------
predictor = DefaultPredictor(cfg)
test_mapper = DatasetMapper(cfg, is_train=False)

# FIXED: correct arguments
eval_dataloader = build_detection_test_loader(
    cfg,
    VAL_DATASET_NAME,
    mapper=test_mapper
)

evaluator = COCOEvaluator(
    VAL_DATASET_NAME,
    cfg,
    False,
    output_dir=os.path.join(cfg.OUTPUT_DIR, "validation_results")
)

# ----------------------------------------------------------
# 4. Run Evaluation
# ----------------------------------------------------------
print("🚀 Starting evaluation...")

results = inference_on_dataset(
    predictor.model,
    eval_dataloader,
    evaluator
)

print("\n================================")
print(" FINAL EVALUATION RESULTS")
print("================================")
print(results)

/home/aamena/Assessment_tcs/.venv/lib/python3.10/site-packages/detectron2/model_zoo/model_zoo.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


[11/19 17:41:51 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /home/aamena/Assessment_tcs/output/domain_a_baseline/model_final.pth ...
[11/19 17:41:51 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
WARNING [11/19 17:41:51 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[11/19 17:41:51 d2.data.datasets.coco]: Loaded 1728 images in COCO format from datasets/DomainB_Enhanced/annotations.coco.json
[11/19 17:41:51 d2.data.build]: Distribution of instances among all 4 categories:
|   category    | #instances   |  category  | #instances   |  category  | #instances   |
|:-------------:|:-------------|:----------:|:-------------|:----------:|:-------------|
| In-"Pa",-"Sc" | 0            |     In     | 1705         |     Pa     | 1240         |
|      Sc       | 1162         |            

/home/aamena/Assessment_tcs/.venv/lib/python3.10/site-packages/torch/functional.py:505: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4317.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
W1119 17:41:51.953000 72591 torch/fx/_symbolic_trace.py:52] is_fx_tracing will return true for both fx.symbolic_trace and torch.export. Please use is_fx_tracing_symbolic_tracing() for specifically fx.symbolic_trace or torch.compiler.is_compiling() for specifically torch.export/compile.


[11/19 17:41:52 d2.evaluation.evaluator]: Inference done 11/1728. Dataloading: 0.0007 s/iter. Inference: 0.0595 s/iter. Eval: 0.0013 s/iter. Total: 0.0616 s/iter. ETA=0:01:45
[11/19 17:41:57 d2.evaluation.evaluator]: Inference done 92/1728. Dataloading: 0.0008 s/iter. Inference: 0.0599 s/iter. Eval: 0.0013 s/iter. Total: 0.0621 s/iter. ETA=0:01:41
[11/19 17:42:02 d2.evaluation.evaluator]: Inference done 173/1728. Dataloading: 0.0008 s/iter. Inference: 0.0599 s/iter. Eval: 0.0012 s/iter. Total: 0.0620 s/iter. ETA=0:01:36
[11/19 17:42:07 d2.evaluation.evaluator]: Inference done 254/1728. Dataloading: 0.0009 s/iter. Inference: 0.0599 s/iter. Eval: 0.0012 s/iter. Total: 0.0620 s/iter. ETA=0:01:31
[11/19 17:42:12 d2.evaluation.evaluator]: Inference done 335/1728. Dataloading: 0.0009 s/iter. Inference: 0.0599 s/iter. Eval: 0.0013 s/iter. Total: 0.0621 s/iter. ETA=0:01:26
[11/19 17:42:17 d2.evaluation.evaluator]: Inference done 416/1728. Dataloading: 0.0009 s/iter. Inference: 0.0600 s/iter. E

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

# ------------------------------------------
# Convert Detectron2 COCO results → DataFrames
# ------------------------------------------
bbox_results = results["bbox"]
segm_results = results["segm"]

bbox_df = pd.DataFrame(bbox_results, index=[0]).T.rename(columns={0: "Score"})
segm_df = pd.DataFrame(segm_results, index=[0]).T.rename(columns={0: "Score"})

print("📌 Bounding Box Metrics")
display(bbox_df)

print("\n📌 Segmentation Metrics")
display(segm_df)

📌 Bounding Box Metrics


,Score
AP,44.689166
AP50,75.688988
AP75,45.271050
APs,5.866369
APm,30.118983
APl,52.207525
"AP-In-""Pa"",-""Sc""",NaN
AP-In,28.806506
AP-Pa,64.237188
AP-Sc,41.023804



📌 Segmentation Metrics


,Score
AP,36.074892
AP50,70.262057
AP75,33.510787
APs,2.667840
APm,20.879162
APl,42.925133
"AP-In-""Pa"",-""Sc""",NaN
AP-In,23.416788
AP-Pa,59.165025
AP-Sc,25.642861
